In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Supponendo che df_copy sia già definito e contenga i dati
# Per esempio:
# df_copy = pd.read_csv('path_to_your_data.csv')

# Selezionare solo le variabili numeriche
df_numeric = log_transformed_df.select_dtypes(include=[float, int])

# Standardizzare i dati prima di applicare la PCA
scaler = StandardScaler()
df_standardized = scaler.fit_transform(df_numeric)

# Applicare la PCA con 4 componenti principali
num_components = 5
pca = PCA(n_components=num_components)
principal_components = pca.fit_transform(df_standardized)

# Creare un DataFrame con le componenti principali
df_pca = pd.DataFrame(data=principal_components, columns=[f'Principal Component {i+1}' for i in range(num_components)])

# Aggiungere i nomi delle variabili originali come indice del DataFrame
df_pca.index = df_numeric.index

# Visualizzare il DataFrame risultante con i nomi delle variabili originali
print(df_pca.head())

# Visualizzare la variabilità spiegata delle variabili
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_df = pd.DataFrame({
    'Principal Component': [f'Principal Component {i+1}' for i in range(num_components)],
    'Explained Variance Ratio': explained_variance_ratio[:num_components]
})

print(explained_variance_df)

# Loadings (senza rotazione)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# Creare un DataFrame per i loadings con i nomi delle variabili
loadings_df = pd.DataFrame(loadings, index=df_numeric.columns, columns=[f'Principal Component {i+1}' for i in range(num_components)])

# Visualizzare i loadings
print(loadings_df)

# Esegui il clustering K-means sui dati trasformati dalla PCA
kmeans = KMeans(n_clusters=8, random_state=42)
clusters = kmeans.fit_predict(principal_components)

# Aggiungi i cluster al DataFrame delle componenti principali
df_pca['Cluster'] = clusters

# Calcola l'SSE (Sum of Squared Errors)
sse = kmeans.inertia_
print(f"SSE (Sum of Squared Errors): {sse}")

# Calcola il Silhouette Score
silhouette_avg = silhouette_score(principal_components, clusters)
print(f"Silhouette Score: {silhouette_avg}")

# Visualizza i risultati del clustering su 3 componenti principali (grafico 3D)
fig_3d = plt.figure(figsize=(10, 8))
ax = fig_3d.add_subplot(111, projection='3d')
scatter = ax.scatter(df_pca['Principal Component 1'], df_pca['Principal Component 2'], df_pca['Principal Component 3'], c=df_pca['Cluster'], cmap='viridis', marker='o')
ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], kmeans.cluster_centers_[:, 2], s=300, c='red', marker='x')
ax.set_title('Clustering K-means dopo PCA')
ax.set_xlabel('Prima componente principale')
ax.set_ylabel('Seconda componente principale')
ax.set_zlabel('Terza componente principale')
plt.show()

# Calcola SSE e Silhouette Score per diversi valori di k per scegliere il numero di cluster
sse_list = []
sil_list = []
k_values = range(2, 21)  # Prova valori di k da 2 a 20

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(principal_components)
    sse_list.append(kmeans.inertia_)
    sil_list.append(silhouette_score(principal_components, clusters))

# Crea la figura con due sottotrame (subplots)
fig = make_subplots(rows=1, cols=2, subplot_titles=("SSE vs k", "Silhouette Score vs k"))

# Grafico SSE vs k
fig.add_trace(
    go.Scatter(x=list(k_values), y=sse_list, mode='lines+markers', name="SSE"),
    row=1, col=1
)
fig.update_xaxes(title_text="k", row=1, col=1)
fig.update_yaxes(title_text="SSE", row=1, col=1)

# Grafico Silhouette Score vs k
fig.add_trace(
    go.Scatter(x=list(k_values), y=sil_list, mode='lines+markers', name="Silhouette"),
    row=1, col=2
)
fig.update_xaxes(title_text="k", row=1, col=2)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)

# Layout generale della figura
fig.update_layout(title_text="SSE e Silhouette Score per il clustering K-Means",
                  showlegend=False)

# Mostra il grafico
fig.show()